# PIC-SURE API use-case: quick analysis on CIBMTR data

This is a tutorial notebook aimed to get the user quickly up and running with the python PIC-SURE API. It covers the main functionalities of the API.

## PIC-SURE python API 
### What is PIC-SURE? 

As part of the BioData Catalyst initiative, the Patient Information Commons Standard Unification of Research Elements (PIC-SURE) platform has been integrating clinical and genomic datasets from multiple TOPMed and TOPMed related studies funded by the National Heart Lung and Blood Institute (NHLBI). 

Original data exposed through the PIC-SURE API encompasses a large heterogeneity of data organization underneath. PIC-SURE hides this complexity and exposes the different study datasets in a single tabular format. By simplifying the process of data extraction, it allows investigators to focus on downstream analysis and to facilitate reproducible science.

### More about PIC-SURE
The API is available in two different programming languages, python and R, enabling investigators to query the databases the same way using either language.

PIC-SURE is a larger project from which the R/python PIC-SURE API is only a brick. Among other things, PIC-SURE also offers a graphical user interface that allows researchers to explore variables across multiple studies, filter patients that match criteria, and create cohorts from this interactive exploration.

The python API is actively developed by the Avillach Lab at Harvard Medical School.

PIC-SURE API GitHub repo:
* https://github.com/hms-dbmi/pic-sure-python-adapter-hpds
* https://github.com/hms-dbmi/pic-sure-python-client

 -------   

# Getting your own user-specific security token

**Before running this notebook, please be sure to review the "Get your security token" documentation, which exists in the NHLBI_BioData_Catalyst [README.md file](https://github.com/hms-dbmi/Access-to-Data-using-PIC-SURE-API/tree/master/NHLBI_BioData_Catalyst#get-your-security-token). It explains about how to get a security token, which is mandatory to access the databases.**

# Environment set-up

### Pre-requisite
- python 3.6 or later
- pip: python package manager, already available in most system with a python interpreter installed ([pip installation instructions](https://pip.pypa.io/en/stable/installing/))

### IPython Magic command

The following code loads the `autoreload` IPython extension. Although `autoreload` is not necessary to execute the rest of the notebook, it enables the notebook to reload every dependency each time python code is executed.  This will allow the notebook to take into account changes in imported external files, such as the user defined functions stored in separate file, without needing to manually reload libraries. This is helpful when developing interactively. Learn more about [IPython Magic commands](https://ipython.readthedocs.io/en/stable/interactive/magics.html).

In [ ]:
%load_ext autoreload
%autoreload 2

### Install Packages

Install the following:
* packages listed in the `requirements.txt` file (listed below, along with verison numbers)
* PIC-SURE API components (from GitHub)
    * PIC-SURE Adapter
    * PIC-SURE Client

In [ ]:
!cat requirements.txt

In [ ]:
import sys
!{sys.executable} -m pip install -r requirements.txt

In [ ]:
!{sys.executable} -m pip install --upgrade --force-reinstall git+https://github.com/hms-dbmi/pic-sure-python-client.git
!{sys.executable} -m pip install --upgrade --force-reinstall git+https://github.com/hms-dbmi/pic-sure-python-adapter-hpds.git
!{sys.executable} -m pip install --upgrade --force-reinstall git+https://github.com/hms-dbmi/pic-sure-biodatacatalyst-python-adapter-hpds.git

Import all the external dependencies and user-defined functions stored in the `python_lib` folder.

In [ ]:
import json
from pprint import pprint

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from scipy import stats

import PicSureClient
import PicSureBdcAdapter

from python_lib.utils import get_multiIndex_variablesDict, get_dic_renaming_vars, joining_variablesDict_onCol

##### Set the display parameters for tables and plots

In [ ]:
# Pandas DataFrame display options
pd.set_option("max.rows", 100)

# Matplotlib parameters options
fig_size = plt.rcParams["figure.figsize"]
 
# Prints: [8.0, 6.0]
fig_size[0] = 14
fig_size[1] = 8
plt.rcParams["figure.figsize"] = fig_size

font = {'weight' : 'bold',
        'size'   : 14}

plt.rc('font', **font)

### Connecting to a PIC-SURE resource

The following is required to get access to data through the PIC-SURE API: 
- a network URL
- a resource id, and 
- a user-specific security token.

If you have not already retrieved your user-specific token, please refer to the "Get your security token" section of the [README.md](https://github.com/hms-dbmi/Access-to-Data-using-PIC-SURE-API/tree/master/NHLBI_BioData_Catalyst#get-your-security-token) file.

In [ ]:
PICSURE_network_URL = "https://biodatacatalyst.integration.hms.harvard.edu/picsure"
#PICSURE_network_URL = "https://picsure.biodatacatalyst.nhlbi.nih.gov/picsure"
resource_id = "02e23f52-f354-4e8b-992c-d37c8b9ba140"
token_file = "token.txt"

In [ ]:
with open(token_file, "r") as f:
    my_token = f.read()

In [ ]:
client = PicSureClient.Client()
connection = client.connect(PICSURE_network_URL, my_token, True)
adapter = PicSureBdcAdapter.Adapter(connection)
resource = adapter.useResource(resource_id)

Two objects are created here: a `connection` and a `resource` object.

Since will only be using a single resource, **the `resource` object is actually the only one we will need to proceed with data analysis hereafter**. 

It is connected to the specific data source ID we specified and enables us to query and retrieve data from this database.

### Getting help with the PIC-SURE python API

Each object exposed by the PicSureBdcHpds library has a `help()` method. Calling it without parameters will print out  information about functionalities of this object.

In [ ]:
resource.help()

For instance, this output tells us that this `resource` object has 3 methods and provides a quick definition of those methods.

## Using the *variables dictionary*

Once a connection to the desired resource has been established, we first need to understand which variables are available in the database. We will use the `dictionary` method of the `resource` object to do this.

A `dictionary` instance enables us to retrieve matching records by searching for a specific term, or to retrieve information about all the available variables, using the `find()` method. For instance, looking for variables containing the term `Sex` in their names is done this way: 

In [ ]:
dictionary = resource.dictionary()
dictionary_search = dictionary.find("Sex")

Objects created by the `dictionary.find` method can expose the search results using 4 different methods: `.count()`, `.keys()`, `.entries()`, and `.DataFrame()`. 

In [ ]:
pprint({"Count": dictionary_search.count(), 
        "Keys": dictionary_search.keys()[0:3],
        "Entries": dictionary_search.entries()[0:3]})

In [ ]:
dictionary_search.DataFrame().head()

**The `.DataFrame()` method enables us to get the result of the dictionary search in a pandas DataFrame format. This way, it allows us to:** 


* Use the various information exposed in the dictionary (patient count, variable type ...) as criteria for variable selection.
* Use the row names of the DataFrame to get the actual variable names to be used in the query, as shown below.

Variable names aren't very pratical to use right away for two reasons:
1. Very long
2. Presence of backslashes that prevent copy-pasting. 

However, retrieving the dictionary search result in the form of a dataframe can help access the variable names. Let's say we want to retrieve every variable from the HCT for SCD dataset in the form of a DataFrame. We can do this using the code below:

In [ ]:
plain_variablesDict = resource.dictionary().find("HCT for SCD").DataFrame()

In [ ]:
plain_variablesDict.iloc[10:20,:]

Using the `dictionary.find()` function without arguments returns every entry, as shown in the help documentation.

In [ ]:
resource.dictionary().help()

The dictionary currently returned by the API provides information about the variables, such as:
- observationCount: number of entries with non-null value
- categorical: type of the variables, True if strings, False if numerical
- min/max: only provided for numerical variables
- HpdsDataType: 'phenotypes' or 'genotypes'. Currently, the API only expsoses'phenotypes' variables

### Export full data dictionary to CSV

In order to export the data dictionary first we will create a Pandas dataframe called `fullVariableDict`.

In [ ]:
fullVariableDict = resource.dictionary().find().DataFrame()

Let's make sure that `fullVariableDict` dataframe contains some values.

In [ ]:
fullVariableDict.iloc[0:3,:]

In [ ]:
fullVariableDict.to_csv('data_dictionary.csv')

You should now see a `data_dictionary.csv` in the Jupyter Hub file explorer, in the same folder as this notebook.

#### Variable dictionary + pandas multiIndex

We can use a simple user-defined function (`get_multiIndex_variablesDict`) to add a little more information to the variable dictionary and to simplify working with variables names. It takes advantage of pandas MultiIndex functionality [see pandas official documentation on this topic](https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html).

Although not an official feature of the API, such functionality illustrates how to quickly select groups of related variables.

Printing the multiIndexed variable Dictionary allows us to quickly see the tree-like organization of the variable names. Moreover, original and simplified variable names are now stored respectively in the `name` and `simplified_name` columns (simplified variable names is simply the last component of the variable name, which is usually the most informative to let us know what each variable is about).

In [ ]:
variablesDict = get_multiIndex_variablesDict(plain_variablesDict)

In [ ]:
variablesDict.loc[["Hematopoietic Cell Transplant for Sickle Cell Disease (HCT for SCD) ( phs002385 )"],:]

In [ ]:
# Limit the number of lines to be displayed for the future outputs
pd.set_option("max.rows", 50)

Below is a simple example to illustrate the ease of use a multiIndex dictionary. Let's say we are interested in the variables within the "5 - CRF data collection track only" of the "Hematopoietic Cell Transplant for Sickle Cell Disease (HCT for SCD) ( phs002385 )" study.

In [ ]:
mask_study = variablesDict.index.get_level_values(0) == "Hematopoietic Cell Transplant for Sickle Cell Disease (HCT for SCD) ( phs002385 )"
mask_dctrack = variablesDict.index.get_level_values(1) == "5 - CRF data collection track only"
dctrack_variables = variablesDict.loc[mask_study & mask_dctrack,:]
dctrack_variables

This simple filter can be easily combined with other filters to quickly select variables of interest.

## Querying and retrieving data

The second cornerstone of the API is the `query` object. It is how we retrieve data from the resource.

First, we need to create a query object.

In [ ]:
my_query = resource.query()

The query object has several methods that enable us to build a query:

| Method | Arguments / Input | Output|
|--------|-------------------|-------|
| query.select.add() | variable names (string) or list of strings | all variables included in the list (no record subsetting)|
| query.require.add() | variable names (string) or list of strings | all variables; only records that do not contain null values for input variables |
| query.anyof.add() | variable names (string) or list of strings | all variables; only records that contain at least one non-null value for input variables |
| query.filter.add() | variable name and additional filtering values | input variable; only records that match filter criteria |

All 4 methods can be combined when building a query. The record eventually returned by the query has to meet all the different specified filters.

### Building the query

Let's say we are interested in the age at which patients from the following cohort received their transplant:
* males
* patients with avascular necrosis
* patients that received their transplant after the year 1999

First we will find variables pertaining to sex and avascular necrosis. We can do this by searching for "Sex" and "Avascular necrosis" in the `simplified_name` column of `variablesDict`.

In [ ]:
sex_var = variablesDict.loc[variablesDict["simplified_name"] == "Sex", "name"].values[0]
avascular_necrosis_varname = variablesDict.loc[variablesDict["simplified_name"] == "Avascular necrosis", "name"].values[0]

In [ ]:
# Peek at the result for avascular necrosis
variablesDict.loc[variablesDict["simplified_name"] == "Avascular necrosis", "name"] 

Next, we can find the variable pertaining to "Year of transplant".

In [ ]:
yr_transplant_varname = variablesDict.loc[variablesDict["simplified_name"] == "Year of transplant", "name"].values[0]
yr_transplant_varname

Now we can create a new query and apply our filters to retrieve the cohort of interest.

In [ ]:
# Patients with avascular necrosis
my_query.select().add(avascular_necrosis_varname)
my_query.filter().add(avascular_necrosis_varname, "Yes")

In [ ]:
# Males
my_query.select().add(sex_var)
my_query.filter().add(sex_var, "Male")

In [ ]:
# Patients receiving transplants after 1999
my_query.select().add(yr_transplant_varname)
my_query.filter().add(yr_transplant_varname, min=2000)

Using this cohort, we can add the variable of interest: "Patient age at transplant, years"

In [ ]:
age_transplant_var = variablesDict.loc[variablesDict["simplified_name"] == "Patient age at transplant, years", "name"].values[0]
my_query.select().add(age_transplant_var)

## Retrieving the data

Once our query object is finally built, we use the `query.run()` function to retrieve the data corresponding to our query

In [ ]:
query_df = my_query.getResultsDataFrame().set_index("Patient ID")

In [ ]:
query_df.head()

Once the data has been retrieved as a dataframe, you can use python functions to conduct analyses and create visualizations, such as this:

In [ ]:
query_df[age_transplant_var].plot.hist(legend=None, 
                                       title= "Age when transplant received in males with arrhythmia from 2000 to present", 
                                       bins=15)